In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import gym
import numpy as np 
import random
from collections import deque

In [3]:
env=gym.make('CartPole-v0')
state=env.reset()
print(state)
print(state.reshape(1,4))
print(type(state))

[-0.01839194 -0.00490567  0.02179601  0.03859984]
[[-0.01839194 -0.00490567  0.02179601  0.03859984]]
<class 'numpy.ndarray'>


In [4]:
env=gym.make('Blackjack-v0')
state=env.reset()
print(state)
print(type(state))
print(len(env.observation_space))
print(env.action_space.n)
s=np.array(state)
print(s)
print(type(s))
# rewards=0
# for _ in range(100):
#     next_state, reward, done, _=env.step(env.action_space.sample())
#     rewards+=reward
#     print(next_state, reward, done)
#     if done:
#         env.reset()
# print('rewards ',rewards)

(12, 10, False)
<class 'tuple'>
3
2
[12 10  0]
<class 'numpy.ndarray'>


In [22]:
class Dualing_DQN():
    def __init__(self,env,dualing):
        self.state_size=len(env.observation_space)
        self.action_size=env.action_space.n
        self.dualing=dualing
        self.memory = deque(maxlen=3000)
        self.epsilon=1.0
        self.epsilon_min=0.01
        self.epsilon_decay = 0.995
        self.train_batch=32
        self.gamma=0.9
        self.create_network()
        

    def create_network(self):
        input=keras.Input(shape=(3,))
        h1=layers.Dense(24, activation='relu')(input)
        h2=layers.Dense(24, activation='relu')(h1)
        v=layers.Dense(1, activation='linear')(h2)
            
        h3=layers.Dense(24, activation='relu')(h1)
        a=layers.Dense(self.action_size, activation='linear')(h3)

        out = v + (a - tf.math.reduce_mean(a, axis=1, keepdims=True))
        self.model = keras.Model(inputs=input, outputs=out)
        self.model.compile(loss='mse',optimizer='adam', metrics=['accuracy'])
        self.model.summary()

    def remember(self,state,action,reward,next_state,done):
        self.memory.append((state,action,reward,next_state,done))

    def predict_action(self,state):
        return self.model.predict(state)
    def act(self,state):
        if random.random() < self.epsilon:
            return random.randrange(self.action_size)
        else:
            #print(self.predict_action(state)) 
            return np.argmax(self.predict_action(state)[0])
    def train(self):
        if len(self.memory)>=self.train_batch:
            minibatch = random.sample(self.memory,self.train_batch) 
            state_batch = np.zeros([self.train_batch,self.state_size])
            target_batch = np.zeros([self.train_batch,self.action_size]) 
            for i,(state, action, reward, next_state, done) in enumerate(minibatch):
                state_batch[i,:] = state
                print(state,state.shape)
                target_batch[i,:] = self.predict_action(state)
                target_batch[i,action] = reward if done else reward+self.gamma*np.amax(self.predict_action(next_state)[0])
                # print(state_batch.shape,target_batch.shape)
            self.model.fit(state_batch, target_batch, epochs=1, verbose=0)
            if self.epsilon > self.epsilon_min:
                self.epsilon *= self.epsilon_decay
agent=Dualing_DQN(env,True)

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 3)]          0                                            
__________________________________________________________________________________________________
dense_31 (Dense)                (None, 24)           96          input_8[0][0]                    
__________________________________________________________________________________________________
dense_34 (Dense)                (None, 24)           600         dense_31[0][0]                   
__________________________________________________________________________________________________
dense_35 (Dense)                (None, 2)            50          dense_34[0][0]                   
____________________________________________________________________________________________

In [23]:
EPISODE = 500 # Episode limitation
STEP = 300 # Step limitation in an episode
TEST=10
# Hyper Parameters for DQN
GAMMA = 0.9 # discount factor for target Q
INITIAL_EPSILON = 0.5 # starting value of epsilon
FINAL_EPSILON = 0.01 # final value of epsilon
REPLAY_SIZE = 10000 # experience replay buffer size
BATCH_SIZE = 32 # size of minibatchEPISODE = 500 # Episode limitation
STEP = 300 # Step limitation in an episode
TEST=10
# Hyper Parameters for DQN
GAMMA = 0.9 # discount factor for target Q
INITIAL_EPSILON = 0.5 # starting value of epsilon
FINAL_EPSILON = 0.01 # final value of epsilon
REPLAY_SIZE = 10000 # experience replay buffer size
BATCH_SIZE = 32 # size of minibatch
saveModelName = './Dueling_DQN'
for times in range(EPISODE):
        state = np.array(env.reset())
        rewards=0
        for i in range(199):
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            next_state=np.array(next_state)
            agent.remember(state,action,reward,next_state,done)
            state = next_state
            if done:
                # print('[times]:{}/{}\t\t[i]:{}\t\t[epsilon]:{}'.format(times,EPISODE,i,agent.epsilon))
                # rewards+=reward
                # print(rewards)
                break
        agent.train()

[[ 8 10  0]] (1, 3)


InternalError:  Blas GEMM launch failed : a.shape=(1, 3), b.shape=(3, 24), m=1, n=24, k=3
	 [[node model_6/dense_31/MatMul (defined at <ipython-input-22-90cc00643ef7>:33) ]] [Op:__inference_distributed_function_1411]

Function call stack:
distributed_function
